# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [12]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# set the parameters to run the script
directories = ['/media/elea/ssd/sdgasf']
calib_directory = '/media/elea/ssd/calib'
    
# set the parameters to be used for the line visualisation
# NB: parameter file accessible in ./src/processingmm/data/parameters_visualisations.json
parameter_set = 'default'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
run_all = True

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes both with PDDN when available and without PDDN
PDDN_mode = 'both'

# Set the wavelengths to be processed
# 1. 'all': processes all the available wavelenght
# 2. [xxx, yyy]: processes only the wavelenghts 'xxx' and 'yyy'
wavelengths = [550, 600]

# Processing mode
# 1. 'no_viz': processes only the MM - no visualization at all. useful for fast computation
# 2. 'default': processes the MM and plots the polarimetric parameters maps (i.e. depolarization, azimuth, 
# retardance, diattenuation, azimuth local variability)
# 3. 'full': do like default, and additionally plot the MM components, as well as the line
# visualization

# define if pdf figures should be saved (takes a lot of time) - no impact when processing_mode is set to no_viz

# NB: processing time without PDDN
# 'no_viz': 0.71s
# 'default', save_pdf_figs False: 2.25s
# 'default', save_pdf_figs True: 3.60s
# 'full', save_pdf_figs False: 3.95s
# 'full', save_pdf_figs True: 8.07s
processing_mode = 'full'
save_pdf_figs = False


parameters = batch_processing.get_parameters(directories, calib_directory, wavelengths, parameter_set = parameter_set, 
                                PDDN_mode = PDDN_mode, processing_mode = processing_mode, run_all = run_all, 
                                save_pdf_figs = save_pdf_figs)

In [19]:
%%time
times = batch_processing.batch_process_master(parameters)
times

processing without PDDN...


  0%|                                                     | 0/1 [00:00<?, ?it/s]

Processing: ['/media/elea/ssd/sdgasf/2024-03-19_T_CC-over-90-30um_FX_M_1']


100%|█████████████████████████████████████████████| 1/1 [00:08<00:00,  8.94s/it]


processing without PDDN done.
processing with PDDN...


  0%|                                                     | 0/1 [00:00<?, ?it/s]

Processing: ['/media/elea/ssd/sdgasf/2024-03-19_T_CC-over-90-30um_FX_M_1']
 >> PDDN Loading Model: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_550_Fresh_HB.pt -- ...
 [wrn] PDDN Model Summary NOT Available!
 >> PDDN Loading Model: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_550_Fresh_HB.pt -- Complete
*** PDDN DENOISING FILTER *** 
 >> PDDN Denoising: total time-points: 1000
 >> PDDN Denoising: start time-point: 1
 >> PDDN Denoising: final time-point: 0
 >> PDDN Denoising: recursive filtering enabled (2 x cascade): False
 >> MPI Denoising PDDN Performance: Elapsed time = 1.423 s
 
 >> Exported X3D .cod file as: ./temp_processing/2024-03-19_T_CC-over-90-30um_FX_M_1/raw_data/550nm/550_Intensite_PDDN.cod
 >> PDDN Loading Model: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_600_Fresh_HB.pt -- ...
 [wrn] PDDN Model Summary NOT Available!
 >> PDDN Load

100%|█████████████████████████████████████████████| 1/1 [00:11<00:00, 11.81s/it]

processing with PDDN done.
CPU times: user 32.8 s, sys: 4.1 s, total: 36.9 s
Wall time: 20.8 s


{'MM_processing': 0.48772692680358887,
 'azimuth_std_processing': 0.011881589889526367,
 'denoising': 0.48879218101501465,
 'save_npz': 0.11351227760314941,
 'full_processing': 3.0581841468811035,
 'plotting': 5.170031785964966,
 'total': 11.819952726364136}

In [20]:
%%time

# Approximate time to process the line visualization save_pdf_figs True : 3.85s
# Approximate time to process the line visualization save_pdf_figs False : 4.92s
times = batch_processing.batch_visualization(parameters)
times

CPU times: user 14.6 s, sys: 1.7 s, total: 16.3 s
Wall time: 13.9 s


13.914740085601807

In [21]:
# align the measurements captured at different wavelengths - could cause issue when using masks obtained
# at 550nm as the images are slightly shifted 
from processingmm.addons import align_wavelengths

PDDN_mode = 'both'
run_all = False
wl_to_align = [600, 650]
align_wavelengths.align_wavelenghts(directories, PDDN_mode, run_all, wl_to_align)

Aligning wavelenght: 600nm


100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 4655.17it/s]


Aligning wavelenght: 650nm


100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 12520.31it/s]


In [22]:
# add the calibration matrices (A.cod and W.cod) to the folder for processing
# needed by Chris
from processingmm.addons import add_calibration
add_calibration.add_calibration(directories)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 15.63it/s]
